<a href="https://www.kaggle.com/code/priyanagda/tf-idf-lr?scriptVersionId=109301254" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score
from scipy.sparse import hstack

In [93]:
sub_path = "../input/goodreads-books-reviews-290312/goodreads_sample_submission.csv"
test_path = "../input/goodreads-books-reviews-290312/goodreads_test.csv"
train_path = "../input/goodreads-books-reviews-290312/goodreads_train.csv"

In [94]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sub_df = pd.read_csv(sub_path)

In [95]:
# train_df.head()

In [96]:
# test_df.head()

In [97]:
train_df.drop(["user_id", "book_id", "review_id", "date_added", "date_updated", "read_at", "started_at"], inplace=True, axis=1)
test_df.drop(["user_id", "book_id", "review_id", "date_added", "date_updated", "read_at", "started_at"], inplace=True, axis=1)

In [98]:
# # delete all rows with column 'rating' has value 0
# index = train_df[ (train_df['rating'] == 0)].index
# train_df.drop(index , inplace=True)

In [99]:
# len(index)

In [100]:
# train_df.reset_index(inplace=True, drop=True)
# train_df.head()

In [101]:
# test_df.head()

In [102]:
train_df.loc[:,'review_text'] = train_df.review_text.apply(lambda x : str.lower(x))
test_df.loc[:,'review_text'] = test_df.review_text.apply(lambda x : str.lower(x))

In [103]:
train_text = train_df['review_text']
test_text = test_df['review_text']
all_text = pd.concat([train_text, test_text])

In [104]:
stopwords = stopwords.words('english')
stopwords = stopwords[:116]
stopwords = stopwords.extend(['d', 'll', 're', 's', 've'])

In [105]:
# from nltk.corpus import stopwords
# stopwords.words('english')

In [106]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stopwords,
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

CPU times: user 6min 46s, sys: 5.8 s, total: 6min 52s
Wall time: 6min 52s


In [107]:
# train_word_features

In [108]:
# np.array(train_df['n_votes']).shape

In [109]:
# train_features = hstack( [train_word_features, np.array(train_df['n_votes']).reshape((-1, 1)), np.array(train_df['n_comments']).reshape((-1, 1))])
# test_features = hstack([test_word_features, np.array(test_df['n_votes']).reshape((-1, 1)), np.array(test_df['n_comments']).reshape((-1, 1))])

In [110]:
train_features = hstack([train_word_features])
test_features = hstack([test_word_features])

In [111]:
# train_features

In [112]:
train_target = train_df['rating']
classifier = LogisticRegression(solver='sag')
classifier.fit(train_features, train_target)
# from sklearn.ensemble import RandomForestClassifier
# classifier = RandomForestClassifier(max_depth=2, random_state=0)
# classifier.fit(train_features, train_target)

LogisticRegression(solver='sag')

In [119]:
test_pred = classifier.predict(test_features)

In [120]:
test_pred

array([4, 4, 4, ..., 5, 5, 5])

In [121]:
sub_df['rating'] = test_pred

In [122]:
sub_df.to_csv("submission.csv", index=False)

In [123]:
sub_df.rating.value_counts()

4    194607
5    140433
3     93797
2     29407
1     10431
0      9358
Name: rating, dtype: int64

In [124]:
train_df.rating.value_counts()

4    313688
5    265007
3    188972
2     72627
0     30988
1     28718
Name: rating, dtype: int64